In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [2]:
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_51260_2024-04-25~2025-04-25.xlsx')
na_df = pd.read_excel('/Users/seanyu_mac/Downloads/릴리약사님 이뮨 빌베리 공구 (2).xlsx')

In [3]:
fs_df = fs_df[fs_df['상품 번호'] == 51260]
fs_df['박스수량'] = fs_df['옵션명'].apply(lambda x: x.split('X')[1].split('박스')[0])
fs_df['박스수량'] = fs_df['박스수량'].astype(int)
fs_df = fs_df[['결제 번호', '주문 상태','결제 완료일', '옵션명', '수량', '박스수량', '정산가']]

In [4]:
fs_df['구분'] = '친한스토어'

In [5]:
temp1 = fs_df[~fs_df['주문 상태'].isin(['배송완료', '배송중', '구매확정'])]
temp1.groupby('주문 상태')['정산가'].sum()

주문 상태
반품완료(반품완료)      65000
반품진행(수거진행)     325000
취소완료          1589000
Name: 정산가, dtype: int64

In [6]:
fs_df.loc[fs_df['주문 상태'].isin(['배송완료', '배송중', '구매확정']), '정산가'].sum()

55886000

In [7]:
56211000 - 55886000

325000

In [8]:
na_df['박스수량'] = na_df['옵션정보'].apply(lambda x: x.split('X')[1].split('박스')[0])
na_df['박스수량'] = na_df['박스수량'].astype(int)
na_df['정산가'] = 0
con1 = na_df['박스수량'] == 4
con2 = na_df['박스수량'] == 6
con3 = na_df['박스수량'] == 8
na_df.loc[con1, '정산가'] = na_df.loc[con1, '수량'] * 96000
na_df.loc[con2, '정산가'] = na_df.loc[con2, '수량'] * 132000
na_df.loc[con3, '정산가'] = na_df.loc[con3, '수량'] * 165000

na_df = na_df[['상품주문번호', '주문상태', '주문일시', '옵션정보', '수량', '박스수량', '정산가']]

In [9]:
na_df['구분'] = '네이버스토어'
na_df.columns = fs_df.columns

In [10]:
temp_df = pd.concat([fs_df, na_df])

In [11]:
temp_df['주문 상태'].unique()

array(['배송완료', '배송중', '구매확정', '취소완료', '반품진행(수거진행)', '반품완료(반품완료)', '취소'],
      dtype=object)

In [12]:
temp_df = temp_df[temp_df['주문 상태'].isin(['결제완료', '배송중', '배송완료', '구매확정'])]

In [13]:
temp_df['주문 상태'].unique()

array(['배송완료', '배송중', '구매확정'], dtype=object)

In [14]:
temp_df['결제 완료일'] = pd.to_datetime(temp_df['결제 완료일'])
temp_df['결제 완료일'] = temp_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))
pivot = pd.pivot_table(temp_df, index=['결제 완료일', '옵션명'], values=['정산가', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합')
data = pd.pivot_table(temp_df, index=['결제 완료일', '옵션명'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')['결제 번호']
pivot['주문수량'] = data
pivot.columns = ['박스수량', '매출', '주문수량']
result = pd.DataFrame(pivot)
result = result[['매출', '주문수량', '박스수량']]
result.applymap(lambda x: f'{x:,}')

매출 주문수량   박스수량
결제 완료일     옵션명                                
2025-04-17 30포X4박스       8,201,000  107    428
           30포X6박스       3,960,000   30    180
           30포X8박스       8,250,000   49    392
2025-04-18 30포X4박스       2,400,000   25    100
           30포X6박스       1,056,000    8     48
           30포X8박스       2,970,000   17    136
2025-04-19 30포X4박스       2,112,000   22     88
           30포X6박스       1,056,000    8     48
           30포X8박스       1,815,000   11     88
           선택: 30포X4박스      96,000    1      4
2025-04-20 30포X4박스       1,728,000   18     72
           30포X6박스         792,000    6     36
           30포X8박스       1,155,000    7     56
2025-04-21 30포X4박스       1,824,000   19     76
           30포X6박스       1,188,000    9     54
           30포X8박스       1,155,000    7     56
           선택: 30포X8박스     330,000    1      8
2025-04-22 30포X4박스       3,168,000   33    132
           30포X6박스       1,320,000   10     60
           30포X8박스       2,475,000   11     88
2025-04-23 30포X4박스       3,552,000   37    148
           30포X6박스       1,584,000   12     72
           30포X8박스       4,125,000   25    200
           선택: 30포X4박스     288,000    3     12
           선택: 30포X6박스     132,000    1      6
           선택: 30포X8박스     330,000    2     16
총합                      57,062,000  479  2,604

In [15]:
pivot = pd.pivot_table(temp_df, index=['결제 완료일', '구분'], values=['정산가', '박스수량'], aggfunc='sum', fill_value=0, margins=True, margins_name='총합')
data = pd.pivot_table(temp_df, index=['결제 완료일', '구분'], values=['결제 번호'], aggfunc='count', fill_value=0, margins=True, margins_name='총합')['결제 번호']
pivot['주문수량'] = data
pivot.columns = ['박스수량', '매출', '주문수량']
result = pd.DataFrame(pivot)
result = result[['매출', '주문수량', '박스수량']]
result.applymap(lambda x: f'{x:,}')

매출 주문수량   박스수량
결제 완료일     구분                            
2025-04-17 친한스토어   20,411,000  186  1,000
2025-04-18 친한스토어    6,426,000   50    284
2025-04-19 네이버스토어      96,000    1      4
           친한스토어    4,983,000   41    224
2025-04-20 친한스토어    3,675,000   31    164
2025-04-21 네이버스토어     330,000    1      8
           친한스토어    4,167,000   35    186
2025-04-22 친한스토어    6,963,000   54    280
2025-04-23 네이버스토어     750,000    6     34
           친한스토어    9,261,000   74    420
총합                 57,062,000  479  2,604

In [48]:
a = temp_df[temp_df['구분'] == '네이버스토어']
b = pd.DataFrame(pd.pivot_table(a, index=['결제 완료일', '옵션명'], values=['정산가', '박스수량'], aggfunc='sum', margins=True, margins_name='합계'))

In [17]:
temp_df = pd.concat([fs_df, na_df])
temp_df['주문 상태'].unique()

array(['배송완료', '배송중', '구매확정', '취소완료', '반품진행(수거진행)', '반품완료(반품완료)', '취소'],
      dtype=object)

In [18]:
temp_df['결제 완료일'] = pd.to_datetime(temp_df['결제 완료일'])
temp_df['결제 완료일'] = temp_df['결제 완료일'].apply(lambda x: x.strftime('%Y-%m-%d'))

cancel_df = temp_df[temp_df['주문 상태'].isin(['취소완료', '취소'])].reset_index(drop=True)
cancel_df.columns = ['결제 번호', '주문 상태', '결제 완료일', '옵션명', '수량', '박스수량', '매출', '구분']
cancel_df['매출'] = cancel_df['매출'].apply(lambda x: f'{x:,}')
cancel_df

,결제 번호,주문 상태,결제 완료일,옵션명,수량,박스수량,매출,구분
0,parma20250423112901isS66,취소완료,2025-04-23,30포X4박스,1,4,"96,000",친한스토어
1,parma20250422181850biYb3,취소완료,2025-04-22,30포X4박스,1,4,"96,000",친한스토어
2,parma20250422170824bcbg6,취소완료,2025-04-22,30포X8박스,1,8,"165,000",친한스토어
3,parma20250422074830bjvw0,취소완료,2025-04-22,30포X4박스,1,4,"96,000",친한스토어
4,parma20250421224649ZnfK8,취소완료,2025-04-21,30포X4박스,1,4,"96,000",친한스토어
5,parma20250421183834bs8K4,취소완료,2025-04-21,30포X4박스,1,4,"96,000",친한스토어
6,parma20250420200944Z1YO4,취소완료,2025-04-20,30포X8박스,1,8,"165,000",친한스토어
7,parma20250419170241ViXX9,취소완료,2025-04-19,30포X4박스,1,4,"96,000",친한스토어
8,parma20250419162902YQvi8,취소완료,2025-04-19,30포X8박스,1,8,"165,000",친한스토어
9,parma202504190956329dWa8,취소완료,2025-04-19,30포X4박스,1,4,"96,000",친한스토어
